# データセットの作成（走行の録画）

### Boardを判定

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7

In [ ]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=30)

In [ ]:
camera.running = False

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import subprocess
import os
import cv2
import time
import threading
import smbus
import json

i2c = smbus.SMBus(I2C_BUSNUM)
addr = 0x08
count = 0

In [ ]:
########################################
# remap
########################################
def remap(x, oMin, oMax, nMin, nMax):
    """
    args:
      x: input value
      oMin: old minimum value
      oMax: old maximum value
      nMin: new minimum value
      nMax: new maximum value
    """

    #range check
    if oMin == oMax:
        print("Warning: Zero input range")
        return None

    if nMin == nMax:
        print("Warning: Zero output range")
        return None

    #check reversed input range
    reverseInput = False
    oldMin = min( oMin, oMax )
    oldMax = max( oMin, oMax )
    if not oldMin == oMin:
        reverseInput = True

    #check reversed output range
    reverseOutput = False   
    newMin = min( nMin, nMax )
    newMax = max( nMin, nMax )
    if not newMin == nMin :
        reverseOutput = True

    portion = (x-oldMin)*(newMax-newMin)/(oldMax-oldMin)
    if reverseInput:
        portion = (oldMax-x)*(newMax-newMin)/(oldMax-oldMin)

    result = portion + newMin
    if reverseOutput:
        result = newMax - portion

    return result

def live():
    global count,running,save_path_widget,task_widget,dataset_widget,i2c,camera,cv2,annotation
    
    save_xy_dir = save_path_widget.value + task_widget.value + "/" + dataset_widget.value + "/xy"
    save_speed_dir = save_path_widget.value + task_widget.value + "/" + dataset_widget.value + "/speed"

    if not os.path.exists(save_xy_dir):
        subprocess.call(['mkdir', '-p', save_xy_dir])

    if not os.path.exists(save_speed_dir):
        subprocess.call(['mkdir', '-p', save_speed_dir])

    write_log("録画を開始しました。")
    count = 0
    num = 0
    while running:
        image = camera.read()
        
        if annotation == True:
            data = i2c.read_i2c_block_data(addr, 0x01, 12)
            xy = data[0] << 24 | data[1] << 16 | data[2] << 8| data[3]
            
            xy = remap(xy, left, right, 0, 224)
            if xy < 0:
                xy = 0
            elif xy > 224:
                xy = 224
            xy_img_name = "{}_{}_{:0=5}.jpg".format(xy,112,count)
            xy_image_path = os.path.join(save_xy_dir, xy_img_name)
            cv2.imwrite(xy_image_path, image)    

            speed = data[4] << 24 | data[5] << 16 | data[6] << 8| data[7]
            speed = remap(speed, stop, front, 0, 224)
            if speed < 0:
                speed = 0
            speed_img_name = "{}_{}_{:0=5}.jpg".format(0,speed,count)
            speed_image_path = os.path.join(save_speed_dir, speed_img_name)
            cv2.imwrite(speed_image_path, image)
            
            #start = data[8] << 24 | data[9] << 16 | data[10] << 8| data[11]
        else:
            if num % 2 == 0:
                img_name = "{}_{}_{:0=5}.jpg".format(0,0,num)
                image_path = os.path.join(save_xy_dir, img_name)
                cv2.imwrite(image_path, image)
            num+=1
        count+=1
    
def record(change):
    global running, execute_thread
    if dataset_widget.value == "":
        write_log("保存先 dataset名を指定してください。")
    else:
        running = True
        execute_thread = threading.Thread(target=live)
        execute_thread.start()

def stop_record(change):
    global count, running, execute_thread
    running = False
    execute_thread.join()
    write_log(str(count) + "枚の画像を生成し、録画を終了しました。")

In [ ]:
try:
    with open('raw_params.json') as f:
        json_str = json.load(f)

        stop = json_str["raw_speed"]["stop"]
        front = json_str["raw_speed"]["front"]
        back = json_str["raw_speed"]["back"]
        left = json_str["raw_steering"]["left"]
        center = json_str["raw_steering"]["center"]
        right = json_str["raw_steering"]["right"]
except:
    print("Don't exit raw_param.json")

In [ ]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
CATEGORIES = ['xy','speed']
TASK = ['camera']
DIR = "./"

category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='category')
dataset_widget = ipywidgets.Text(description='dataset')
task_widget = ipywidgets.Dropdown(options=TASK, description='task')
start_button = ipywidgets.Button(description='Record')
stop_button = ipywidgets.Button(description='Stop')
annotation_box = ipywidgets.Checkbox(False, description='Auto Annotation')

start_button.on_click(record)
stop_button.on_click(stop_record)

save_path_widget = ipywidgets.Text(description='folder')
save_path_widget.value = DIR

annotation = False
def on_annotation(c):
    global annotation
    annotation = True

annotation_box.observe(on_annotation)

data_collection_widget = ipywidgets.VBox([
    save_path_widget,
    task_widget,
    dataset_widget,
    annotation_box,
    ipywidgets.HBox([start_button, stop_button]),
    process_widget
])

display(data_collection_widget)

### カメラの終了処理(必須)

In [ ]:
import time
camera.running = False
time.sleep(1)
camera.cap.release()